In [1]:
from conllu import parse

In [70]:
import random
import pandas as pd
import re
from collections import Counter, defaultdict
import csv

In [43]:
p = re.compile('[а-яА-ЯёЁ]')
f = open('../marked/linguistics.conll', encoding='utf-8')
data = f.read()
tree = parse(data)
words=[]
words_only=[]
for i in tree:
    for j in i:
        if j['form']!='_' and j['form'].isalpha() and (j['upostag']=='S' or j['upostag']=='V') and p.search(j['form']) and j['form'] not in '[]\/':
            for w in j['form'].lower().split():
                words.append((w, j['upostag']))
                words_only.append(w)
print(words[:10])

[('введение', 'S'), ('поляризацией', 'S'), ('традиции', 'S'), ('было', 'V'), ('называть', 'V'), ('свойство', 'S'), ('языковых', 'S'), ('единиц', 'S'), ('заключается', 'V'), ('невозможности', 'S')]


In [4]:
df = pd.read_excel('linguistics_collocation_counts.xlsx','bigrams')


In [37]:
pairs = set()
coll = df['ngram'].tolist()
len_words_2 = len(words)//2
coll_tags = df['tag'].tolist()
good_bigrams = []
good_tags = []
for i in range(len(coll)):
    pair = coll[i].split()
    pair_tags = coll_tags[i].split()
    if (pair_tags[0]=='S' or pair_tags[0]=='V') and (pair_tags[1]=='S' or pair_tags[1]=='V') and len(pair[0])>1 and len(pair[1])>1:       
        good_bigrams.append((pair[0],pair[1]))
        good_tags.append((pair_tags[0],pair_tags[1]))
    pairs.add((words[random.randrange(len_words_2)],words[random.randrange(len_words_2)]))
bigrams = pairs - set(coll)
bigrams = list(bigrams)
print(bigrams[:10])
bad_tags, bad_bigrams = [],[]
for bigr in bigrams:
    bad_tags.append((bigr[0][1], bigr[1][1]))
    bad_bigrams.append((bigr[0][0], bigr[1][0]))
len_good_bigrams = len(good_bigrams)
bad_bigrams = bad_bigrams[:len_good_bigrams]
bad_tags = bad_tags[:len_good_bigrams]

[(('строить', 'V'), ('помнится', 'V')), (('валентность', 'S'), ('говорится', 'V')), (('говорит', 'V'), ('дается', 'V')), (('может', 'V'), ('стимул', 'S')), (('степашина', 'S'), ('был', 'V')), (('половины', 'S'), ('задач', 'S')), (('оркестрион', 'S'), ('получают', 'V')), (('явление', 'S'), ('стороны', 'S')), (('результате', 'S'), ('журналистом', 'S')), (('искандер', 'S'), ('проверив', 'V'))]


In [67]:
labels_good, labels_bad = [1 for c in good_bigrams], [0 for b in bad_bigrams]
print(len(labels_good), len(labels_bad))
train_test_data = pd.DataFrame(
{
    'bigrams':good_bigrams+bad_bigrams,
    'tags':good_tags+bad_tags,
    'labels':labels_good + labels_bad
})
print(train_test_data)


15194 15194
                             bigrams  labels    tags
0         (абессив, существительных)       1  (S, S)
1          (абзаца, характеризуются)       1  (S, V)
2                (абрахам, указывал)       1  (S, V)
3                (абрахама, формами)       1  (S, S)
4      (абстрактности, конкретности)       1  (S, S)
5           (абсцисс, соответствуют)       1  (S, V)
6                  (аванесов, пишет)       1  (S, V)
7               (август, употреблял)       1  (S, V)
8                  (автобуса, стоял)       1  (S, V)
9              (автомобиль, потерял)       1  (S, V)
10         (автопереводе, прибегает)       1  (S, V)
11               (автор, апеллирует)       1  (S, V)
12                     (автор, было)       1  (S, V)
13                    (автор, видит)       1  (S, V)
14                 (автор, выделяет)       1  (S, V)
15                 (автор, выражает)       1  (S, V)
16                   (автор, данных)       1  (S, S)
17                (автор, допускае

In [74]:
train_test_data.to_csv('data.csv')

In [44]:
def count_weirdness(lemmas_counts,contrast_lemmas_counts,flag):
    Ts, Tg = sum(lemmas_counts.values()), sum(contrast_lemmas_counts.values())
    weirdnesses = defaultdict(float)
    for key in lemmas_counts.keys():
        Ws, Wg = lemmas_counts[key], contrast_lemmas_counts[key]
        weirdnesses[key] = (Ws/Ts)/((Wg+Ws)/(Tg+Ts))
    result = sorted(weirdnesses.items(), key=lambda k_v: k_v[1], reverse=True)
    with open('result_{0}grams.txt'.format(flag),'w',encoding='utf-8') as res:
        for item in result:
            res.write(str(item)+'\r\n')

In [46]:
lemmas_counts = Counter(words_only)
contrast_lemmas = []
with open('corpus_analyzed_1.txt', 'r', encoding='utf-8') as contrast_corpus:
    for line in contrast_corpus:
        line = line.strip()
        splitted = line.split("\t")
        if len(splitted) > 1:
            contrast_lemmas.append(splitted[0])
contrast_lemmas_counts = Counter(contrast_lemmas)
flag = 'uni'  # униграммы
count_weirdness(lemmas_counts, contrast_lemmas_counts, flag)